
# ResNet18 model
### Packages

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report, top_k_accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import time
from sklearn.model_selection import train_test_split

### Model Training

In [ ]:
# Define transforms for data augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define transforms for validation and test (no augmentation)
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset without transforms
folder_path = '/user/5/toubalih/Lab_project_2/Lab_project_S8/updated_png/'
dataset = ImageFolder(root=folder_path)

# Extract the targets for stratified splitting
targets = np.array([sample[1] for sample in dataset.samples])

# Calculate dataset sizes
train_size = 0.7
val_size = 0.15
test_size = 0.15

# First split: train+val and test
train_val_idx, test_idx, y_train_val, y_test = train_test_split(
    range(len(targets)), targets, stratify=targets, test_size=test_size, random_state=42)

# Calculate the validation size with respect to the remaining data (train+val)
val_size_adjusted = val_size / (train_size + val_size)

# Second split: train and val
train_idx, val_idx, y_train, y_val = train_test_split(
    train_val_idx, y_train_val, stratify=y_train_val, test_size=val_size_adjusted, random_state=42)

# Create Subsets
train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)
test_dataset = Subset(dataset, test_idx)

# Apply transforms to each subset
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform

# Define data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the model architecture (using a pre-trained model like ResNet as an example)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))

# Define loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 10
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Track start time
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct_train / total_train * 100
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = correct_val / total_val * 100
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    # Update learning rate
    scheduler.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# Plot accuracy evolution after training
plt.figure(figsize=(10, 5))
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Evolution')
plt.legend()
plt.grid(True)
plt.show()

# Final evaluation on the test set
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = test_correct / test_total * 100
print(f"Final Test Accuracy: {test_accuracy:.2f}%")

# Calculate and print total run time
end_time = time.time()
total_time = end_time - start_time
print(f'Total run time: {total_time:.2f} seconds')

### Confusion Matrix and Top Accuracy

In [ ]:

# Plot confusion matrix and metrics
def plot_confusion_matrix_and_metrics(model, device, data_loader, class_names):
    """
    Generates and plots a confusion matrix and prints classification metrics for the given data.
    """
    model.eval()
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            preds = torch.argmax(outputs, dim=1)
            true_labels.extend(target.cpu().numpy())
            pred_labels.extend(preds.cpu().numpy())

    # Compute the confusion matrix
    conf_mat = confusion_matrix(true_labels, pred_labels)
    # Compute other classification metrics
    class_report = classification_report(true_labels, pred_labels, target_names=class_names)

    # Print the classification report
    print("Classification Report:")
    print(class_report)

    # Plot the confusion matrix
    plt.figure(figsize=(15, 15))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix for ResNet18')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

# Example of how to call the function
#plot_confusion_matrix_and_metrics(model, device, test_loader, class_names)

# Evaluate model with top-k accuracy
def evaluate_model_with_sklearn_top_k(model, device, test_loader, classes, k_list=[1, 5]):
    """
    Evaluates the model on the test set using scikit-learn's top_k_accuracy_score for multiple k values.

    Parameters:
    - model (torch.nn.Module): The trained model to evaluate.
    - device (torch.device): The device (CPU/GPU) on which to perform the evaluation.
    - test_loader (torch.utils.data.DataLoader): DataLoader for the test set.
    - classes (list): List of all class labels.
    - k_list (list): List of top-k values to calculate.

    Returns:
    - None, prints the average Top-K accuracy for each k in k_list.
    """
    model.eval()  # Ensure the model is in evaluation mode
    true_labels = []
    all_scores = []

    with torch.no_grad():  # Disable gradient computation
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            # Softmax to convert model logits to probabilities
            probabilities = torch.softmax(outputs, dim=1)
            all_scores.extend(probabilities.cpu().numpy())
            true_labels.extend(target.cpu().numpy())

    true_labels = np.array(true_labels)
    all_scores = np.array(all_scores)  # Shape: (n_samples, n_classes)

    # Ensure the labels parameter is correctly handled
    labels = np.arange(len(classes))  # Assuming classes correspond to [0, num_classes-1]

    for k in k_list:
        top_k_accuracy = top_k_accuracy_score(true_labels, all_scores, k=k, labels=labels)
        print(f"Top-{k} Accuracy on Test Set: {top_k_accuracy * 100:.2f}%")

# Example usage
evaluate_model_with_sklearn_top_k(model, device, test_loader, class_names, k_list=[1, 5])

### Accuracy and Loss Curve

In [ ]:

# Create a new figure
fig, ax1 = plt.subplots()

# Plot training and validation accuracy
color = 'tab:blue'
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy', color=color)
ax1.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy', color=color, marker='o')
ax1.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', linestyle='dashed', color=color, marker='o')
ax1.tick_params(axis='y', labelcolor=color)
ax1.legend(loc='upper left')
ax1.grid(True)

# Create a second y-axis to plot training and validation loss
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Loss', color=color)
ax2.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', color=color, marker='x')
ax2.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', linestyle='dashed', color=color, marker='x')
ax2.tick_params(axis='y', labelcolor=color)
ax2.legend(loc='upper right')

plt.title('Accuracy and Loss Curve of ResNet(50 dataset)')
plt.tight_layout()
plt.show()
